In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from finta import TA
import tensorflow as tf
from tensorflow.keras.models import Sequential
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from keras.regularizers import l2
from sklearn.model_selection import KFold

In [2]:
# Load data
MSFT_path = Path("../files/MSFT.csv")
VIX_path = Path("../files/^VIX.csv")
FEDFUNDS_path = Path("../files/FEDFUNDS (1).csv")
SPY_index = Path("../files/SPY.csv")

In [3]:
msft_df = pd.read_csv(MSFT_path, index_col="Date")
fear_index_df = pd.read_csv(VIX_path, index_col="Date").rename(columns={"Close": "Fear_index"})
spy_index_df = pd.read_csv(SPY_index, index_col="Date").rename(columns={"Close": "SPY_index"})
fedfunds_df = pd.read_csv(FEDFUNDS_path, index_col="DATE")

In [4]:
# Convert index to datetime index
fedfunds_df.index = pd.to_datetime(fedfunds_df.index)

# Resample the fed_funds_df to have daily frequency and forward fill the values
fedfunds_df_monthly = fedfunds_df.resample('D').ffill()


In [5]:
# Concatenate dataframes
concatenated_df = pd.concat([msft_df, fear_index_df['Fear_index'], spy_index_df['SPY_index']], axis=1)
concatenated_df.index = pd.to_datetime(concatenated_df.index)


In [6]:
# Merge with fedfunds_df
concatenated_df = pd.merge(concatenated_df, fedfunds_df_monthly, left_index=True, right_index=True)


In [7]:
# Drop rows with NaN values
concatenated_df = concatenated_df.dropna()


In [8]:
# Shift target variable
concatenated_df['Target'] = concatenated_df['Close'].shift(-5)
concatenated_df = concatenated_df.dropna()

In [9]:
concatenated_df.head(10)

,Open,High,Low,Close,Adj Close,Volume,Fear_index,SPY_index,FEDFUNDS,Target
2014-04-30,40.400002,40.500000,40.169998,40.400002,34.210922,35458700.0,13.41,188.309998,0.09,39.419998
2014-05-01,40.240002,40.360001,39.950001,40.000000,33.872189,28787400.0,13.25,188.330002,0.09,39.639999
2014-05-02,40.310001,40.340000,39.660000,39.689999,33.609688,43416600.0,12.91,188.059998,0.09,39.540001
2014-05-05,39.520000,39.639999,39.299999,39.430000,33.389515,22460900.0,13.29,188.419998,0.09,39.970001
2014-05-06,39.290001,39.349998,38.950001,39.060001,33.076210,27112400.0,13.80,186.779999,0.09,40.419998
2014-05-07,39.220001,39.509998,38.509998,39.419998,33.381042,41744500.0,13.40,187.880005,0.09,40.240002
2014-05-08,39.340000,39.900002,38.970001,39.639999,33.567348,32120400.0,13.43,187.679993,0.09,39.599998
2014-05-09,39.540001,39.849998,39.369999,39.540001,33.482643,29647600.0,12.92,187.960007,0.09,39.830002
2014-05-12,39.740002,40.020000,39.650002,39.970001,33.846786,22782600.0,12.23,189.789993,0.09,39.750000
2014-05-13,39.919998,40.500000,39.849998,40.419998,34.469315,27004800.0,12.13,189.960007,0.09,39.680000


In [10]:
# Calculate technical indicators using finta
data = concatenated_df.copy()  # Use the existing DataFrame concatenated_df
data['MA'] = TA.SMA(data, 20)  # 20-period Simple Moving Average
data['RSI'] = TA.RSI(data, 14)  # 14-period RSI

# Calculate Bollinger Bands correctly
bb_bands = TA.BBANDS(data, 20, 2)

# Assign Bollinger Bands values to DataFrame columns
data['BB_UPPER'] = bb_bands['BB_UPPER']
data['BB_MIDDLE'] = bb_bands['BB_MIDDLE']
data['BB_LOWER'] = bb_bands['BB_LOWER']

# Convert index to datetime
data.index = pd.to_datetime(data.index)

# Display the calculated technical indicators
data.tail()


,Open,High,Low,Close,Adj Close,Volume,Fear_index,SPY_index,FEDFUNDS,Target,MA,RSI,BB_UPPER,BB_MIDDLE,BB_LOWER
2024-03-18,414.250000,420.730011,413.779999,417.320007,417.320007,20106000.0,14.33,512.859985,5.33,422.859985,410.386003,56.917294,422.869983,410.386003,397.902023
2024-03-19,417.829987,421.670013,415.549988,421.410004,421.410004,19837900.0,13.82,515.710022,5.33,421.649994,411.317003,59.820193,424.187046,411.317003,398.446960
2024-03-20,422.000000,425.959991,420.660004,425.230011,425.230011,17860100.0,13.04,520.479980,5.33,421.429993,412.469504,62.370449,426.005438,412.469504,398.933569
2024-03-21,429.829987,430.820007,427.160004,429.369995,429.369995,21296200.0,12.92,522.200012,5.33,420.720001,413.355504,64.965751,428.844429,413.355504,397.866578
2024-03-22,429.700012,429.859985,426.070007,428.739990,428.739990,17636500.0,13.06,521.210022,5.33,424.570007,414.275504,64.239658,431.135412,414.275504,397.415595


In [11]:
# Define features and target
X = concatenated_df.drop("Close", axis=1)
y = concatenated_df["Close"]

In [12]:
data.drop(columns=['Open', 'High', 'Low', 'Close', 'Adj Close'], inplace=True)


In [13]:
# Display the modified DataFrame
data.head()

,Volume,Fear_index,SPY_index,FEDFUNDS,Target,MA,RSI,BB_UPPER,BB_MIDDLE,BB_LOWER
2014-04-30,35458700.0,13.41,188.309998,0.09,39.419998,NaN,NaN,NaN,NaN,NaN
2014-05-01,28787400.0,13.25,188.330002,0.09,39.639999,NaN,0.0,NaN,NaN,NaN
2014-05-02,43416600.0,12.91,188.059998,0.09,39.540001,NaN,0.0,NaN,NaN,NaN
2014-05-05,22460900.0,13.29,188.419998,0.09,39.970001,NaN,0.0,NaN,NaN,NaN
2014-05-06,27112400.0,13.80,186.779999,0.09,40.419998,NaN,0.0,NaN,NaN,NaN


In [14]:
data_clean = data.dropna()
data_clean.index.rename('date', inplace=True)
data_clean.to_csv('../clean_data/MSFT_prepared_data.csv', index=True)

In [15]:
data_clean.head()

,Volume,Fear_index,SPY_index,FEDFUNDS,Target,MA,RSI,BB_UPPER,BB_MIDDLE,BB_LOWER
date,,,,,,,,,,
2014-05-28,25711500.0,11.68,191.380005,0.09,40.320000,39.872,49.481417,40.609894,39.872,39.134105
2014-05-29,19888200.0,11.57,192.369995,0.09,41.209999,39.869,55.366983,40.598293,39.869,39.139706
2014-05-30,34567600.0,11.40,192.679993,0.09,41.480000,39.916,63.657327,40.788030,39.916,39.043969
2014-06-02,18504300.0,11.58,192.899994,0.10,41.270000,39.971,60.625587,40.918503,39.971,39.023496
2014-06-03,18068900.0,11.87,192.800003,0.10,41.110001,40.014,51.773943,40.935837,40.014,39.092162


In [16]:
# Define date cutoff for data split
date_cutoff = "2022-04-30"

# Split data
X_train = X[X.index <= date_cutoff]
X_test = X[X.index > date_cutoff]
y_train = y[y.index <= date_cutoff]
y_test = y[y.index > date_cutoff]

In [17]:
# Scale data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [18]:
# Define the number of folds
k = 10

# Initialize lists to store R-squared scores
train_r2_scores = []
test_r2_scores = []

# Initialize KFold
kf = KFold(n_splits=k, shuffle=True)

# Define the model architecture
model = Sequential([
    Dense(units=16, activation='relu', kernel_regularizer=l2(0.0005), input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.6),
    Dense(units=8, activation='relu', kernel_regularizer=l2(0.0005)),
    Dropout(0.6),
    Dense(units=1)
])

# Example: Train with a smaller learning rate
from keras.optimizers import Adam
adam = Adam(learning_rate=0.0001)  # Adjust learning rate as needed
model.compile(optimizer=adam, loss='mean_squared_error')
model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

# Compile the model
model.compile(optimizer=adam, loss='mean_squared_error')

# Perform k-fold cross-validation
for train_index, test_index in kf.split(X_train_scaled):
    X_train_cv, X_test_cv = X_train_scaled[train_index], X_train_scaled[test_index]
    y_train_cv, y_test_cv = y_train[train_index], y_train[test_index]
    
    
    # Evaluate the model on training data
    train_predictions = model.predict(X_train_cv)
    train_r2 = r2_score(y_train_cv, train_predictions)
    train_r2_scores.append(train_r2)
    
    # Evaluate the model on test data
    test_predictions = model.predict(X_test_cv)
    test_r2 = r2_score(y_test_cv, test_predictions)
    test_r2_scores.append(test_r2)

# Calculate average R-squared scores
avg_train_r2 = np.mean(train_r2_scores)
avg_test_r2 = np.mean(test_r2_scores)

print("Average R-squared (Train):", avg_train_r2)
print("Average R-squared (Test):", avg_test_r2)

/Users/kd17/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 643us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 537us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 397us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 509us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 399us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 592us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 350us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 520us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 345us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 570us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 382us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 519us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 397us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 560us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 360us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 519us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 377us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 527us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 341us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 542us/step
Average R-squared (Train): -2.1543456765871856
Average R-squared (Test): -2.16964251078983


In [19]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Initialize lists to store metrics for selected models
selected_train_r2 = []
selected_test_r2 = []
selected_train_mae = []
selected_train_mse = []
selected_test_mae = []
selected_test_mse = []

# Perform k-fold cross-validation
for train_index, test_index in kf.split(X_train_scaled):
    X_train_cv, X_test_cv = X_train_scaled[train_index], X_train_scaled[test_index]
    y_train_cv, y_test_cv = y_train[train_index], y_train[test_index]
    
    # Define a new model for each fold
    model_fold = Sequential([
        Dense(32, activation='relu', input_shape=(X_train_cv.shape[1],)),
        Dropout(0.5),  # Dropout layer with a dropout rate of 0.5
        Dense(16, activation='relu'),
        Dropout(0.5),  # Dropout layer with a dropout rate of 0.5
        Dense(1)  # Output layer
    ])
    
    # Compile the model with Adam optimizer and mean squared error loss
    model_fold.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
    
    # Train the model
    model_fold.fit(X_train_cv, y_train_cv, epochs=50, batch_size=32, verbose=0)
    
    # Evaluate the model on training data
    train_predictions = model_fold.predict(X_train_cv)
    train_r2 = r2_score(y_train_cv, train_predictions)
    train_mae = mean_absolute_error(y_train_cv, train_predictions)
    train_mse = mean_squared_error(y_train_cv, train_predictions)
    
    # Evaluate the model on test data
    test_predictions = model_fold.predict(X_test_cv)
    test_r2 = r2_score(y_test_cv, test_predictions)
    test_mae = mean_absolute_error(y_test_cv, test_predictions)
    test_mse = mean_squared_error(y_test_cv, test_predictions)
    
    # Append metrics to the lists
    selected_train_r2.append(train_r2)
    selected_test_r2.append(test_r2)
    selected_train_mae.append(train_mae)
    selected_train_mse.append(train_mse)
    selected_test_mae.append(test_mae)
    selected_test_mse.append(test_mse)

# Print metrics for selected models
for idx, (train_r2, test_r2, train_mae, train_mse, test_mae, test_mse) in enumerate(zip(selected_train_r2, selected_test_r2, selected_train_mae, selected_train_mse, selected_test_mae, selected_test_mse), start=1):
    print(f"Model {idx} - Train R-squared: {train_r2}, Test R-squared: {test_r2}, Train MAE: {train_mae}, Train MSE: {train_mse}, Test MAE: {test_mae}, Test MSE: {test_mse}")


/Users/kd17/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 653us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 619us/step


/Users/kd17/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 586us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 558us/step


/Users/kd17/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 573us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 470us/step


/Users/kd17/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 589us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 567us/step


/Users/kd17/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 571us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 552us/step
Model 1 - Train R-squared: 0.9346607972885933, Test R-squared: 0.9481969717500947, Train MAE: 17.70455156828764, Train MSE: 470.3415225971947, Test MAE: 16.563639058972043, Test MSE: 416.42593900569364
Model 2 - Train R-squared: 0.9519980206235856, Test R-squared: 0.9456720312990798, Train MAE: 15.6156126405071, Train MSE: 352.307951621, Test MAE: 15.908422338076754, Test MSE: 365.6139605130205
Model 3 - Train R-squared: 0.9536554219045528, Test R-squared: 0.9545719067757947, Train MAE: 15.140655129150796, Train MSE: 336.6986249819606, Test MAE: 15.072651236439317, Test MSE: 338.1087174641223
Model 4 - Train R-squared: 0.9460651458399407, Test R-squared: 0.9402500115973123, Train MAE: 16.2504608817245, Train MSE: 392.53681977213137, Test MAE: 16.99372593422344, Test MSE: 437.02969864181716
Model 5 - Train R-squared: 0.9586250073350715, Test R-squared: 0.9513825968779297, Train MAE: 14.45159259207197, Train 

In [20]:
# Filter models where both Train R-squared and Test R-squared are less than 0.96
filtered_indices = [i for i, (train_r2, test_r2) in enumerate(zip(selected_train_r2, selected_test_r2)) if train_r2 < 0.96 and test_r2 < 0.96]

# Calculate the absolute difference between train R-squared and test R-squared values for filtered models
abs_diff_r2_filtered = np.abs(np.array(selected_train_r2)[filtered_indices] - np.array(selected_test_r2)[filtered_indices])

# Find the index of the model with the smallest absolute difference among filtered models
best_model_index = filtered_indices[np.argmin(abs_diff_r2_filtered)]

# Retrieve the metrics for the best model
best_train_r2 = selected_train_r2[best_model_index]
best_test_r2 = selected_test_r2[best_model_index]
best_train_mae = selected_train_mae[best_model_index]
best_train_mse = selected_train_mse[best_model_index]
best_test_mae = selected_test_mae[best_model_index]
best_test_mse = selected_test_mse[best_model_index]

# Print metrics for the best model
print(f"Best Model - Train R-squared: {best_train_r2}, Test R-squared: {best_test_r2}, Train MAE: {best_train_mae}, Train MSE: {best_train_mse}, Test MAE: {best_test_mae}, Test MSE: {best_test_mse}")


Best Model - Train R-squared: 0.9326896417532439, Test R-squared: 0.9330738676854374, Train MAE: 17.90752078631239, Train MSE: 491.3705469252869, Test MAE: 17.11367213072455, Test MSE: 476.49953418486245


In [21]:
# Scale data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Define the model architecture
model = Sequential([
    Dense(units=16, activation='relu', kernel_regularizer=l2(0.0005), input_shape=(X_scaled.shape[1],)),
    Dropout(0.6),
    Dense(units=8, activation='relu', kernel_regularizer=l2(0.0005)),
    Dropout(0.6),
    Dense(units=1)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model on the entire dataset
model.fit(X_scaled, y, epochs=50, batch_size=32, verbose=0)

# Predict sequentially on each data point
all_predictions = model.predict(X_scaled)

# Ensure the number of predictions matches the original dataset
assert len(all_predictions) == len(X_scaled)

# Create a DataFrame to store the actual and predicted values
predictions_df = pd.DataFrame({'Actual': y, 'Predicted': all_predictions.flatten()}, index=X.index)

# Ensure index uniqueness in both the original dataset and predictions DataFrame
data_clean_unique_index = data_clean.index.drop_duplicates()
predictions_df = predictions_df.loc[data_clean_unique_index]

# Display the DataFrame
predictions_df


/Users/kd17/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 549us/step


,Actual,Predicted
date,,
2014-05-28,40.009998,37.866375
2014-05-29,40.340000,39.350262
2014-05-30,40.939999,35.868446
2014-06-02,40.790001,39.828880
2014-06-03,40.290001,39.905861
...,...,...
2024-03-18,417.320007,324.626434
2024-03-19,421.410004,325.602600
2024-03-20,425.230011,328.380066


In [22]:
predictions_df.to_csv('../predicted_data/MSFT_predicted_data.csv', index=True)

In [23]:
# Check for NaN values in the predictions DataFrame
nan_values = predictions_df.isnull().sum().sum()

if nan_values == 0:
    print("No NaN values found in the predictions DataFrame.")
    print(predictions_df)
else:
    print(f"Found {nan_values} NaN values in the predictions DataFrame. Please check your data or model.")


No NaN values found in the predictions DataFrame.
                Actual   Predicted
date                              
2014-05-28   40.009998   37.866375
2014-05-29   40.340000   39.350262
2014-05-30   40.939999   35.868446
2014-06-02   40.790001   39.828880
2014-06-03   40.290001   39.905861
...                ...         ...
2024-03-18  417.320007  324.626434
2024-03-19  421.410004  325.602600
2024-03-20  425.230011  328.380066
2024-03-21  429.369995  329.704285
2024-03-22  428.739990  331.442688

[2473 rows x 2 columns]


In [24]:
# Calculate the percentage difference between actual and predicted values
predictions_df['Percentage Difference (%)'] = ((predictions_df['Predicted'] - predictions_df['Actual']) / predictions_df['Actual']) * 100

# Display the DataFrame with percentage difference
predictions_df


,Actual,Predicted,Percentage Difference (%)
date,,,
2014-05-28,40.009998,37.866375,-5.357718
2014-05-29,40.340000,39.350262,-2.453491
2014-05-30,40.939999,35.868446,-12.387769
2014-06-02,40.790001,39.828880,-2.356265
2014-06-03,40.290001,39.905861,-0.953438
...,...,...,...
2024-03-18,417.320007,324.626434,-22.211629
2024-03-19,421.410004,325.602600,-22.734962
2024-03-20,425.230011,328.380066,-22.775896


In [25]:
# Calculate the absolute percentage difference for each data point
predictions_df['Abs_Percentage_Diff'] = abs((predictions_df['Actual'] - predictions_df['Predicted']) / predictions_df['Actual']) * 100

# Calculate the average percentage difference
avg_percentage_diff = predictions_df['Abs_Percentage_Diff'].mean()

print("Average Percentage Difference (%):", avg_percentage_diff)


Average Percentage Difference (%): 26.85409146778956
